In [ ]:
import pandas as pd
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
sys.path.append('..')
from credentials import Credentials

In [ ]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

# Connecting

In [ ]:
# conecting to environment
session.use_role('SYSADMIN')
session.use_database('EXTERNAL_ACCESS_DB')
session.use_schema('PUBLIC')
session.use_warehouse('EXTERNAL_ACCESS_WH')

# Creating a stage that will hold the stored procedure

In [ ]:
stage_name = 'STORED_PROC_STAGE'
session.sql(f'CREATE OR REPLACE STAGE {stage_name}').collect()

# Creating a table

In [ ]:
df1 = session.create_dataframe([[1, 'This is some text that will be translate'], [2, 'One of my travels was in a place call Geneva']], schema=["ROWNUM", "TEXT_TO_TRANSLATE"])
df1.write.save_as_table("TABLE_TO_TRANSLATE", mode="overwrite", table_type="transient")

## Creating Stored procedure

In [ ]:
database_name = 'EXTERNAL_ACCESS_DB'
schema_name = 'PUBLIC'
table_name = 'TABLE_TO_TRANSLATE'
column_to_translate_name = 'TEXT_TO_TRANSLATE'

In [ ]:
def sp_translate_table(session: Session, database_name: str, schema_name: str, table_name: str, column_to_translate_name: str, target_language: str) -> str:
	import _snowflake
	import requests as rq
	import json
	
	table_to_translate = session.sql(f"SELECT * FROM {database_name}.{schema_name}.{table_name}").collect()
	table_translated = []

	token = _snowflake.get_oauth_access_token('cred')
	url = "https://translation.googleapis.com/language/translate/v2"
  	
	for row in table_to_translate:
		sentence = row[column_to_translate_name]
		
		data = {'q': sentence,'target': target_language}

		response = rq.session.post(url, json = data, headers = {"Authorization": "Bearer " + token})
		
		table_translated.append([row[0], response.json()['data']['translations'][0]['translatedText']])
	
	df1 = session.create_dataframe(table_translated, schema=["rownum", "text_translated"])
	df1.write.save_as_table("TABLE_TRANSLATED", mode="overwrite", table_type="transient")

	return 'SUCCESS'

In [ ]:
session.sproc.register(
	func = sp_translate_table,
	name = "sp_translate_table",
	packages = ["snowflake-snowpark-python", "requests"],
	is_permanent = True,
	stage_location = "@EXTERNAL_ACCESS_DB.PUBLIC.STORED_PROC_STAGE",
	source_code_display = False,
	execute_as = 'owner',
	replace = True,
	external_access_integrations = ['google_apis_access_integration'],
	secrets = {'cred': 'oauth_token'}
)

In [ ]:
session.call("sp_translate_table", database_name, schema_name, table_name, column_to_translate_name, 'es')

In [ ]:
session.close()